In [2]:
import cv2
import glob
import sys
dest = "./images/test2/{0}"

def resizeImage(src):
    files = glob.glob('./images/test/{0}/*'.format(src))
    for file in files:
        delim = "/"
        name = file.split(delim)[-1]
        name = file.split("\\")[-1]
        I = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        I = cv2.resize(I, (48, 48), interpolation = cv2.INTER_AREA)
        cv2.imwrite(dest.format(src) + delim + name, I)
resizeImage('angry')
resizeImage('happy')
resizeImage('neutral')
resizeImage('sad')
resizeImage('surprise')

In [3]:
import cv2
import glob
import sys
import random
dest1 = "./images/jaffe/{0}/*"
dest2 = "./images/other_dataset/{0}/*"
dest3 = "./images/other_train/{0}"
dest4 = "./images/other_validation/{0}"
dest5 = "./images/test/{0}"
def copyImage(src, files, dest):
    for file in files:
        delim = "/"
        name = file.split(delim)[-1]
        name = file.split("\\")[-1]
        I = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        cv2.imwrite(dest.format(src) + delim + name, I)
def divide_train_val_test(src):
    files_ja = glob.glob(dest1.format(src))
    files_oth = glob.glob(dest2.format(src))
    random.shuffle(files_ja)
    random.shuffle(files_oth)
    train_ja = files_ja[int(len(files_ja) * .0) : int(len(files_ja) * .7)]
    train_oth = files_oth[int(len(files_oth) * .0) : int(len(files_oth) * .7)]
    val_ja = files_ja[int(len(files_ja) * .7) : int(len(files_ja) * .9)]
    val_oth = files_oth[int(len(files_oth) * .7) : int(len(files_oth) * .9)]
    test_ja = files_ja[int(len(files_ja) * .9) : int(len(files_ja) * 1.0)]
    test_oth = files_oth[int(len(files_oth) * .9) : int(len(files_oth) * 1.0)]
#    copyImage(src, train_ja, dest3)
#    copyImage(src, train_oth, dest3)
#    copyImage(src, val_ja, dest4)
    copyImage(src, val_oth, dest4)
#     copyImage(src, test_ja, dest5)
#    copyImage(src, test_oth, dest5)
divide_train_val_test('angry')
divide_train_val_test('happy')
divide_train_val_test('neutral')
divide_train_val_test('sad')
divide_train_val_test('surprise')

In [6]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.regularizers import l2

earlystopping = EarlyStopping(monitor='val_acc', mode='max', min_delta=0.004)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose =1)
model_check = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min')
train_data_dir = 'images/train'
validation_data_dir = 'images/validation'
train_jaffe_dir = 'images/jaffe_train'
validation_jaffe_dir = 'images/jaffe_validation'
train_other_dir = 'images/other_train'
validation_other_dir = 'images/other_validation'
_dir = 'images/validation'
datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=45, width_shift_range=0.3, height_shift_range=0.1)

train_generator = datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

valid_generator = datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)



model = Sequential()
model.add(Conv2D(64, kernel_size = (3, 3), input_shape = (48, 48, 1), activation = 'relu', kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add((Dropout(0.4)))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add((Dropout(0.5)))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add((Dropout(0.5)))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(5, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(generator=train_generator,
                    steps_per_epoch=4096,
                    validation_data=valid_generator,
                    validation_steps=1024,
                    
                    epochs=1000,
                    callbacks= [model_check, earlystopping, reduce_lr]
)


Found 24865 images belonging to 5 classes.
Found 6104 images belonging to 5 classes.
Epoch 1/1000
4096/4096 [==============================] - 278s 68ms/step - loss: 1.4554 - acc: 0.3690 - val_loss: 1.1685 - val_acc: 0.5197
Epoch 2/1000
4096/4096 [==============================] - 272s 66ms/step - loss: 0.9977 - acc: 0.6114 - val_loss: 0.9091 - val_acc: 0.6494
Epoch 3/1000
4096/4096 [==============================] - 280s 68ms/step - loss: 0.8705 - acc: 0.6704 - val_loss: 0.8479 - val_acc: 0.6765
Epoch 4/1000
4096/4096 [==============================] - 284s 69ms/step - loss: 0.8061 - acc: 0.6979 - val_loss: 0.8284 - val_acc: 0.6859
Epoch 5/1000
4096/4096 [==============================] - 283s 69ms/step - loss: 0.7616 - acc: 0.7165 - val_loss: 0.8386 - val_acc: 0.6767


In [7]:
import numpy as np
from keras.models import load_model
bm = load_model('best_model.h5')
valid_generator.batch_size = 1
sum = np.array([0.0, 0.0])
for i in range(10):
    score = bm.evaluate_generator(generator=valid_generator, steps=832, pickle_safe=False)
    sum += score

print(sum / 10)

C:\Users\Asus\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `evaluate_generator` call to the Keras 2 API: `evaluate_generator(generator=<keras_pre..., steps=832, use_multiprocessing=False)`
  import sys


[0.83669641 0.68064904]


In [ ]:
import pandas as pd
import numpy as np
test_data_dir = 'images/test'
test_generator = datagen.flow_from_directory(
    directory = test_data_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=139,
    class_mode=None,
    shuffle=False,
    seed=42
)
test_generator.reset()
pred = model.predict_generator(test_generator, steps= 1, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
print(predicted_class_indices)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
print(np.array(filenames).shape)
print(np.array(predictions).shape)
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("r.csv",index=False)

In [2]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
emotion_class = {0:'angry', 1:'happy', 2:'neutral', 3:'sad', 4:'surprise'}
best_model = load_model('best_model.h5')

def detect_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (48, 48), interpolation = cv2.INTER_AREA)
    face = face.reshape(1, 48, 48, 1)
    face = face.astype(np.float32)
    face = face / 255
    emotion_index = best_model.predict_classes(face)

    return emotion_class[emotion_index[0]]
    

Using TensorFlow backend.


In [5]:
import cv2


FACE_DETECTOR = cv2.CascadeClassifier('./lbp/cascade.xml')
CAP = cv2.VideoCapture(0)
w = int(CAP.get(cv2.CAP_PROP_FRAME_WIDTH))  
h = int(CAP.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter('out.avi',fourcc, 30.0, (w,h))

FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
FONT_COLOR = (100, 200, 100)
LINE_TYPE = 2

while 1:
    RET, I = CAP.read()

    G = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    FACES = FACE_DETECTOR.detectMultiScale(G, 1.1, 3)

    for (x, y, w, h) in FACES:
        cv2.rectangle(I, (x - 10, y - 30), (x + w + 10, y + h + 30), (255, 100, 100), 3)
        Face = I[y:y+h, x:x+h]
        title = detect_emotion(Face)
        cv2.putText(I, title,
                    (x+10, y-5),
                    FONT,
                    FONT_SCALE,
                    FONT_COLOR,
                    LINE_TYPE)

    cv2.imshow('I', I)
    KEY = cv2.waitKey(60)
    out.write(I)
    if KEY == ord('q'):
        break


CAP.release()
out.release()
cv2.destroyAllWindows()